### 5.4 통계 정보나 특정 규칙에 기반한 추천
- 아래와 같은 통계 정보나 규칙에 기반해 추천하는 것을 생각해보자
  - 직전 1개월의 총 매출 수, 열람 수, 사용자에 따른 평가값의 평균 등 서비스의 데이터 통계 정보를 사용해 아이템을 나열해서 사용자에게 추천
  - 아이템 가격이나 크기와 같이 특정 속성 순서로 나열해 사용자에게 추천
  - 사용자의 나이 등과 같은 특정 속성 정보에 기반해 다른 아이템 추천

- 통계 정보나 아이템 속성 정보에 기반한 추천은 기본적으로 개인화를 수행하지 않는 알고리즘 (구현하기가 비교적 쉬움)
- 단순한 알고리즘을 사용한 추천은 아이템이 어떤 구조로 추천되는지 알기 쉽다는 특징이 있음
  - 사용자가 추천 이유를 쉽게 안다면 구매 행동과 연결되는 경우가 있음
- 사용자 속성 정보로 다른 아이템을 추천하는 경우, 사용자를 몇 가지 세그먼트로 나누고 각각의 사용자 세그먼트에 적합하게 추천하는 방법으로 진행
- **데모그래픽 필터링** : 인구 통계학적 데이터(사용자의 나이, 성별, 거주지 등)에 기반해 아이템을 추천하는 것
- 데모그래픽 필터링 주의점
  - 사용자에 따라 데모그래픽 정보를 입력하지 않거나 잘못된 정보를 기입할 경우
  - **공평성(fairness)** 관점 (사용자에게 성별을 뭍는 것 자체가 문제의 소지가 있음)
- MovieLens 데이터셋을 사용해 사용자들이 과거에 남긴 평가값 중 값이 높은 순서로 추천하는 예시를 보자

In [1]:
# 부모 폴더의 경로 추가
import sys; sys.path.insert(0, '.')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [3]:
# Movielens 데이터 로딩
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='./data/ml-10M100K/')
movielens = data_loader.load()

In [4]:
import numpy as np

# 평갓값이 높은 영화 확인
movie_stats = movielens.train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})
movie_stats.sort_values(by=('rating', 'mean'), ascending=False).head()

/var/folders/hw/qxlf94851g5g0y_m9j7wxzj80000gn/T/ipykernel_31878/3872795521.py:4: FutureWarning: The provided callable <function mean at 0x10d0e1760> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_stats = movielens.train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})


rating     
                                                              size mean
movie_id title                                                         
4354     Unlawful Entry (1992)                                   1  5.0
27255    Wind Will Carry Us, The (Bad ma ra khahad bord)...      1  5.0
7306     Herod's Law (La Ley de Herodes) (2000)                  1  5.0
55814    Diving Bell and the Butterfly, The (Le Scaphand...      2  5.0
3473     Jonah Who Will Be 25 in the Year 2000 (Jonas qu...      1  5.0

- 평가값이 5인 영화가 나열되었지만 평가 수가 적어 5점의 평가가 상위였을 가능성도 있음
- 평가 수가 적으면 신뢰성이 낮으므로 임곗값을 도입해 일정 이상의 평가 수가 있는 영화로 필터링

In [5]:
# 임곗값을 도입
movie_stats = movielens.train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})
atleast_flg = movie_stats['rating']['size'] >= 100
movies_sorted_by_rating = movie_stats[atleast_flg].sort_values(by=('rating', 'mean'), ascending=False)
movies_sorted_by_rating.head()

/var/folders/hw/qxlf94851g5g0y_m9j7wxzj80000gn/T/ipykernel_31878/1864623655.py:2: FutureWarning: The provided callable <function mean at 0x10d0e1760> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_stats = movielens.train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})


rating          
                                                       size      mean
movie_id title                                                       
318      Shawshank Redemption, The (1994)               423  4.492908
50       Usual Suspects, The (1995)                     332  4.459337
912      Casablanca (1942)                              163  4.444785
904      Rear Window (1954)                             129  4.441860
2019     Seven Samurai (Shichinin no samurai) (1954)    104  4.408654

- 평가 수가 100건 이상인 영화로 필터링하면 '쇼생크 탈출', '카사블랑카' 등의 영화가 상위로 올라 납득할 수 있는 결과가 나옴
- 임곗값을 정하면 추천 결과가 달라짐. 실무에서는 임곗값을 1, 10, 100 등으로 패턴을 시험하여 정성적으로 가장 설득력 있는 값을 채택
  - 집계 기간과 다양성을 함꼐 고려해야 함 (상위에 오는 아이템 변동이 없어짐)
  - 임곗값이 너무 높아도 같은 상위에 오는 아이템 변동이 없어짐
- 평갓값이 높은 순의 추천 시스템 성능이 어느 정도인지 측정해보자
  - (src.popularity.py 참고)
- 평가 수의 임곗값을 100으로 시험해보면 RMSE=1.082, Precision@K=0.008, Recall@K=0.027로 무작위 추천일 때의 성능에 비해 그 수치가 높아짐
- 평가 수의 임곗값을 1로 하면 RMSE=1.082, Precision@K=0.000, Recall@K=0.000
- 평가 수의 임곗값을 200으로 하면 RMSE=1.082, Precision@K=0.013, Recall@K=0.042
- 추천 시스템의 성능을 정량적으로 측정함으로써 적절한 임곗값을 설정할 수 있음
  - (RMSE가 변하지 않는 것은 임곗값에 따라 아이템의 평갓값 자체는 해당 아이템 평균 평갓값으로서 계산값이 변하지 않기 때문)

In [6]:
# 인기도 추천
from src.popularity import PopularityRecommender
recommender = PopularityRecommender()
recommend_result = recommender.recommend(movielens, minimum_num_rating=100)

/Users/jeremyshin/dev/recommender/01.추천시스템_입문/src/popularity.py:15: FutureWarning: The provided callable <function mean at 0x10d0e1760> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_rating_average = dataset.train.groupby("movie_id").agg({"rating": np.mean})
/Users/jeremyshin/dev/recommender/01.추천시스템_입문/src/popularity.py:25: FutureWarning: The provided callable <function mean at 0x10d0e1760> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_stats = dataset.train.groupby("movie_id").agg({"rating": [np.size, np.mean]})


In [7]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=1.089, Precision@K=0.008, Recall@K=0.025


In [8]:
# 임곗값을 변경했을 때의 동작
for minimum_num_rating in [1, 200]:
    recommend_result = recommender.recommend(movielens, minimum_num_rating=minimum_num_rating)
    metrics = metric_calculator.calc(
        movielens.test.rating.tolist(), recommend_result.rating.tolist(),
        movielens.test_user2items, recommend_result.user2items, k=10)
    print(metrics)

/Users/jeremyshin/dev/recommender/01.추천시스템_입문/src/popularity.py:15: FutureWarning: The provided callable <function mean at 0x10d0e1760> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_rating_average = dataset.train.groupby("movie_id").agg({"rating": np.mean})
/Users/jeremyshin/dev/recommender/01.추천시스템_입문/src/popularity.py:25: FutureWarning: The provided callable <function mean at 0x10d0e1760> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_stats = dataset.train.groupby("movie_id").agg({"rating": [np.size, np.mean]})
/Users/jeremyshin/dev/recommender/01.추천시스템_입문/src/popularity.py:15: FutureWarning: The provided callable <function mean at 0x10d0e1760> is currently using SeriesGroupBy.mean. In a future version of pandas, the prov

rmse=1.089, Precision@K=0.000, Recall@K=0.000
rmse=1.089, Precision@K=0.013, Recall@K=0.040


/Users/jeremyshin/dev/recommender/01.추천시스템_입문/src/popularity.py:25: FutureWarning: The provided callable <function mean at 0x10d0e1760> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_stats = dataset.train.groupby("movie_id").agg({"rating": [np.size, np.mean]})


## 5.5 연관 규칙
- 연관 규칙(association rule) : 대량의 구매 이력 데이터로부터 아이템 A와 아이템 B는 동시에 구밉하는 경우가 많다는 규칙
  - 귀저귀와 맥주 사례가 가장 유명함
  - 계산 방법이 간단하고 SQL로도 간단히 구현 가능하여 널리 이용
- 연관 규칙 중요 개념 3가지
  - 지지도(support)
  - 확신도(conficence)
  - 리프트값(lift)
- 아래의 표로 설명

  ![구매 이력 데이터](./images/tbl_5-3.png)

  - 사용자 4, 아이템 3개의 구매 이력
  - 활용 예시1) 웹페이지 열람 데이터에 연관 규칙 적용 시 열람 수에 임곗갓을 설정해 특정 횟수 이상에만 적용
  - 활용 예시2) 사용자 단위 집계가 아닌 세션 단위 집계 수행

### 5.5.1 지지도
- 어떤 아이템이 전체 중에서 출현한 비율
- 위의 표를 기준으로 계산하면 다음과 같음
  - 지지도(A) = (A의 출현 수) / 전체 데이터 수=3/4=0.75
  - 지지도(B) = (B의 출현 수) / 전체 데이터 수=3/4=0.75
  - 지지도(C) = (C의 출현 수) / 전체 데이터 수=2/4=0.5
- 아이템 A와 아이템 B가 동시에 출현하는 경우는 아래와 같이 계산 가능
  - 지지도(A and B) = (A와 B의 동시 출현 수) / 전체 데이터 수=2/4=0.75

### 5.5.2 확신도
- 아이템 A가 나타났을 때 아이템 B가 나타날 비율
  - 확신도(A => B) = (A와 B의 동시 출현 수)/(A의 출현 수)=3/3=1.0
  - 이 때 A를 조건부(antecedents), B를 귀결부(consequents)

### 5.5.3 리프트값
- 리프트값 : 아이템 A와 아이템 B의 출현이 어느 정도 상관관계를 갖는지 나타내는 것으로 다음과 같이 정의
  - 리프트(A => B) = 지지도(A and B)/(지지도(A)*지지도(B))=0.75/(0.75*0.75)=1.333
  - 아이템 A와 아이템 B가 나타나는 방법이 서로 전혀 관계가 없고 독립적이면 리프트값은 1이 됨
  - 각 아이템 간 음의 상관관계가 있다면 1보다 작아지고 양의 상관관계가 있다면 1보다 커짐
  - 같은 아이템 (프린터 A와 프린터 B)의 경우 동시 판매가 적으므로 1보다 작아질 것
- 이 리프트 값은 리프트 값에 로그를 취하면 **점별 상호정보량(Pointwise Mutual Information, PMI)** 이 됨
  - word2vec 알고리즘에서 PMI를 요소로 하는 행렬을 행렬 분해한 것으로 알려져 있음
- 3개 이상의 리프트값도 계산 가능
  - 리프트((A and B) => C) = 지지도(A and B and C) / (지지도(A and B)*지지도(C))

### 5.5.4 Apriori 알고리즘을 활용한 고속화
- 아이템 수나 사용자 수가 늘어나면 리프트 값으로 계산할 수 없는 순간이 옴
- 이를 보완한 알고리즘이 Apriori 알고리즘임
- Apriori 알고리즘에서는 지지도가 일정 이상인 아이템이나 아이템 조합만 계산 대상으로 하여 빠르게 계산
  - 추천 시스템에서는 해당 임곗값이 중요한 파라미터임
  - 임곗값을 너무 높이면 일부 인기 아이템만 추천, 너무 낮추면 계산이 무거워지고 노이즈가 많아짐
- MoviLens 데이터로 연관 분석을 진행해보자
  - 여기서는 mlxtend라는 파이썬 라이브러리 사용
- mlxtend 라이브러리에 입력가능하도록 데이터를 행렬 형식으로 변환

In [2]:
# 부모 폴더의 경로를 추가
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [3]:
# Movielens 데이터 로딩
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='./data/ml-10M100K/')
movielens = data_loader.load()

In [10]:
# 사용자 x 영화 행렬 형식으로 변환한다
user_movie_matrix = movielens.train.pivot(index='user_id', columns='movie_id', values='rating')

# 라이브러리를 사용하기 위해 4 이상의 평갓값은 1, 4 미만의 평갓값과 결손값은 0으로 한다
user_movie_matrix[user_movie_matrix < 4] = 0
user_movie_matrix[user_movie_matrix.isnull()] = 0
user_movie_matrix[user_movie_matrix >= 4] = 1

user_movie_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,62000,62113,62293,62344,62394,62801,62803,63113,63992,64716
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 평갓값이 4 이상인 위치가 1, 그 이외가 0이되는 사용자x영화의 행렬이 됨
- 이 데이터를 mlxtend에 입력해서 지지도를 계산

In [12]:
!pip install mlxtend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [13]:
from mlxtend.frequent_patterns import apriori

# 지지도가 높은 영화를 표시
freq_movies = apriori(
    user_movie_matrix, min_support=0.1, use_colnames=True)
freq_movies.sort_values('support', ascending=False).head()

/Users/jeremyshin/.pyenv/versions/3.11.3/envs/recommender/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
42,0.415,(593)
23,0.378,(318)
21,0.368,(296)
19,0.362,(260)
25,0.320,(356)


In [14]:
# movie_id=593의 제목 확인(양들의 침묵)
movielens.item_content[movielens.item_content.movie_id == 593]

,movie_id,title,genre,tag
587,593,"Silence of the Lambs, The (1991)","[Crime, Horror, Thriller]","[based on a book, anthony hopkins, demme, psyc..."


- movie_id=593인 영화는 '양들의 침묵', 사용자의 약 40%가 4 이상의 평가
- 다음으로 이 지지도를 기반으로 리프트값을 계산

In [15]:
from mlxtend.frequent_patterns import association_rules
# 어소시에이션 규칙 계산(리프트 값이 높은 순으로 표시)
rules = association_rules(freq_movies, metric='lift', min_threshold=1)
rules.sort_values('lift', ascending=False).head()[['antecedents', 'consequents', 'lift']]

,antecedents,consequents,lift
658,(5952),(4993),5.459770
659,(4993),(5952),5.459770
1455,"(1291, 260)","(1196, 1198)",4.669188
1454,"(1196, 1198)","(1291, 260)",4.669188
1452,"(1291, 1196)","(260, 1198)",4.171359


- antecedents가 조건부, consequents가 귀결부
- 관계성이 높은 영화 조합을 추출 할 수 있음
  - movie_id=5952(반지의 제왕 시리즈의 1편), movie_id=4993(반지의 제왕 시리즈 2편)
  - 리프트값은 대칭성이 있어 반대의 경우도 같은 값을 가짐
  - 연관 규칙에서 제시되는 것은 상관 관계가 높은 조합이며 인과 관계를 나타내는 것은 아님
  - 반지의 제왕 시리즈 2편을 보고 있는 사람에게는 1편을 추천해도 큰 의미는 없음
- 다음은 이 리프트 값을 사용해 사용자에게 추천해보자
  - 연관 규칙을 사용한 추천 방법은 몇가지가 있음
  - 여기서는 간단히 사용자가 가장 최근 별 4개 이상으로 평가한 영화 5편을 연관 입력으로 사용
  - 5편 중 1편이라도 조건부에 포함되는 연관 규칙을 모두 열거
  - 그 규칙을 리프트값으로 정렬하고 사용자가 과거에 평가한 영화를 제거한 뒤 상위 10편을 사용자에게 추천
- src.association.py 파일 참고

In [16]:
# 어소시에이션 추천
from src.association import AssociationRecommender
recommender = AssociationRecommender()
recommend_result = recommender.recommend(movielens)

/Users/jeremyshin/.pyenv/versions/3.11.3/envs/recommender/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [17]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.011, Recall@K=0.035


In [18]:
# min_support와 정밀도의 관계
for min_support in [0.06, 0.07, 0.08, 0.09, 0.1, 0.11]:
    recommend_result = recommender.recommend(movielens, min_support=min_support)
    metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
    print(metrics)

/Users/jeremyshin/.pyenv/versions/3.11.3/envs/recommender/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


rmse=0.000, Precision@K=0.015, Recall@K=0.049


/Users/jeremyshin/.pyenv/versions/3.11.3/envs/recommender/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


rmse=0.000, Precision@K=0.014, Recall@K=0.043


/Users/jeremyshin/.pyenv/versions/3.11.3/envs/recommender/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


...

rmse=0.000, Precision@K=0.013, Recall@K=0.040


/Users/jeremyshin/.pyenv/versions/3.11.3/envs/recommender/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


rmse=0.000, Precision@K=0.011, Recall@K=0.035


/Users/jeremyshin/.pyenv/versions/3.11.3/envs/recommender/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


rmse=0.000, Precision@K=0.010, Recall@K=0.034


- 결과는 Precision@K=0.011, Recall@K=0.035임
  - 무작위로 추천하는 것보다는 높지만 인기순에 비해 Recall@K 값이 다소 좋지 않음
- 연관 규칙에는 여러 파라미터가 있으며 그 파라미터들을 조정함으로써 정확도를 높일 수 있음
  - min_support 임곗값을 적절히 설정하면 인기도보다 높은 값이 됨
  - min_support=0.06일떄는 rmse=0.000, Precision@K=0.015, Recall@K=0.049
  - min_support가 작을수록 계산에 포함되는 아이템 수가 늘어나기 때문에 계산 시간 증가
  - 실무에서 사용할 때는 계산 속도도 고려해 최적의 임곗값을 결정